# Electric vehicle bidding functions

## To do

This code represents electric vehicles (EVs) which are participating in a real-time market (every 5min). At the beginning of each market interval, the EV places a buy bid (maximum price at which it is willing to buy). Then, the market reveals the true real-time price. If the price is lower than the buy bid, the EV charges. 

In an extension, the EV will also place a supply bid (minimum price at which it is willing to sell), which we call vehicle-to-grid (V2G). If the market price is higher than the sell price, the EV discharges. 

The objective of this task is to write bidding functions which maximize the EV driver's utility or minimizes his or her cost.

Please go through the following steps:

Week 3:
* Go through the code and try to understand each part. Currently, a very simple bidding function (simple_bid()) is implemented. Run the code from the top.
* Write the function calculate_EV_cost() which calculates the costs from charging the EV.
* Write the function visualize_EV_dispatch() which visualizes the EV dispatch as well as the market price.
* Test your code for a week-long simulation period.

Week 4:
* Think of other evaluations and visualizations of the EV charging behavior and implement them.
* Make a strategy of how to implement one of the following new bidding functions and discuss them with Marie-Louise and Lynne:
    - bid_costminizing()
    - bid_costminimizing_V2G()
    - bid_optimal_flexibility()
* Implement your bidding function and test and visualize it.

Week 5:
* Repeat the same for the other bidding functions.

Once you have finalized your code, we will implement your code in GridLAB-D and test them in a more complex distribution system. For the next steps, have a look at the GoogleDoc with your workplan-

When you have questions, don't hesitate to contact Anna, Lynne, or Marie-Louise!

## Modules

In [56]:
import pandas
pandas.options.mode.chained_assignment = None
import numpy as np
#import cvxpy
from matplotlib.dates import drange
import datetime

## Define bidding functions

In [57]:
def bid_simple(dt_sim_time,df_EV_state,df_WS,df_prices):
    prec = 3 # rounding
    
    # Price bid
    mean_p = df_WS.loc[dt_sim_time:(dt_sim_time+datetime.timedelta(hours = 4))]['DA'].mean()
    #df_EV_state['p_sell'] = mean_p / df_EV_state['efficiency']
    df_EV_state['p_buy'] = mean_p * df_EV_state['efficiency']
    
    # Quantity depends on SOC and u
    safety_fac = 0.99
    df_EV_state['residual_b'] = round((3600./interval)*(safety_fac*df_EV_state['SOC_max'] - df_EV_state['SOC_t']),prec) #Recalculate to kW
    df_EV_state['q_buy'] = df_EV_state[['residual_b','u_max']].min(axis=1) #in kW
    df_EV_state['q_buy'].loc[df_EV_state['q_buy'] < 0.1] = 0.0
    
    # For V2G only
    #df_EV_state['residual_s'] = round((3600./interval)*(df_EV_state['SOC_t'] - df_EV_state['SOC_min']),prec) #Recalculate to kW
    #df_EV_state['q_sell'] = df_EV_state[['residual_s','u_max']].min(axis=1) #in kW / only if fully dischargeable
    #df_EV_state['q_sell'].loc[df_EV_state['q_sell'] < 0.1] = 0.0
    
    #print(df_EV_state)
    return df_EV_state


In [58]:
def bid_costminizing(df_EV_state): # Adjust input
    time_horizon = 4*3600/interval # optimization horizon
    # Determine the optimal dispatch schedule of the EV over the optimization horizon,
    #    given day-ahead (DA) prices from df_WS. Use the package cvxpy.
    # Determine profits from (t+1) on if battery had charged/discharged/not done anything in t
    # The profit/cost from charging/doing nothing/discharging in t and the future value in (t+1) 
    #    if the value of the action charging/doing nothing/discharging in t
    #    --> This principle is called the Bellman principle
    # Which prices make the battery operator equivalent between:
    # - charging or doing nothing? --> This is the maximum price at which the battery operator
    #   is willing to buy
    # - discharging or doing nothing? --> This is the minimum price at which the battery operator
    #   is willing to sell
    return df_battery_state

In [59]:
def bid_costminimizing_V2G(df_EV_state):
    # Here, EV essentially works as a battery, with a charging target at departure
    # Check with your co-worker for insights
    return df_battery_state

In [60]:
def bid_optimal_flexibility(df_EV_state):
    # Design and implement a bidding strategy of an agent who values a high state-of-charge
    # but is willing to go below the minimum SOC if the agent could achieve substantial profits
    return df_battery_state

## Evaluation functions

In [80]:
def calculate_EV_cost():
    # Calculate cost of charging during session
    return cost

In [77]:
def visualize_EV_dispatch():
    # Visualize EV dispatch over time as well as the market price
    return

## Other functions

In [63]:
# Updates charging EVs and randomly generated arrival and departure times
def update_EV_rnd(dt_sim_time,df_EV_state):
    df_EV_state['u_t'] = 0.0
    for EV in df_EV_state.index:
        next_event = df_EV_state['next_event'].loc[EV]
        # Change of state (connect or disconnect) ?
        if next_event <= dt_sim_time: #event change in the last period -> change in status of battery
            # previously disconnected / just arrived
            if df_EV_state['connected'].loc[EV] == 0:
                #Set EV
                next_u = df_EV_state['u_max'].loc[EV]
                soc_t = np.random.uniform(0.2,0.8) # randomly generate SOC
                i_max = 1000*next_u/df_EV_state['v_max'].loc[EV]
                #Connect EV
                df_EV_state.at[EV,'connected'] = 1 #Does this one not work such that delay in connection?
                df_EV_state.at[EV,'soc_t'] = soc_t
                df_EV_state.at[EV,'SOC_t'] = soc_t*df_EV_state['SOC_max'].loc[EV]
                u = (df_EV_state['SOC_max'].loc[EV] - df_EV_state['SOC_t'].loc[EV])*(3600./interval) #hypothetical u for constant charging during interval
                df_EV_state.at[EV,'u_t'] = min(u,df_EV_state['u_max'].loc[EV])
                #Departure time
                df_EV_state.at[EV,'next_event'] = dt_sim_time + pandas.Timedelta(hours=np.random.choice(range(3)),minutes=np.random.choice(range(60))) #Next event: disconnection
            # previously connected / just left
            else: #if next event associated with non-NaN SOC - now connected and pot. charging
                #Set df_EV_state   
                df_EV_state.at[EV,'connected'] = 0
                df_EV_state.at[EV,'soc_t'] = 0.0
                df_EV_state.at[EV,'SOC_t'] = 0.0
                #Arrival time today
                df_EV_state.at[EV,'next_event'] = dt_sim_time + pandas.Timedelta(hours=np.random.choice(range(3)),minutes=np.random.choice(range(60))) #Next event: connection
        # During charging event: Update EV state (SOC)
        elif df_EV_state['connected'].loc[EV] == 1:
            #Calculate possible charging rate and stop charging if around 0
            u = (df_EV_state['SOC_max'].loc[EV] - df_EV_state['SOC_t'].loc[EV])*(3600./interval) #hypothetical u for constant charging during interval
            u_t = min(u,df_EV_state['u_max'].loc[EV])
            if u_t < 0.01:
                u_t = 0.0
            df_EV_state.at[EV,'u_t'] = u_t
        # During EV being disconnected : do nothing
        else:
              pass
    return df_EV_state

In [64]:
# Write bids to result tables
def submit_bids_EV(dt_sim_time,retail,df_bids,df_supply_bids,df_buy_bids):
    for ind in df_bids.index:
        if df_bids['q_buy'].loc[ind] > 0.0:
            #retail.buy(df_bids['q_buy'].loc[ind],df_bids['p_buy'].loc[ind],active=df_bids['active_t-1'].loc[ind],appliance_name=ind)
            df_buy_bids = df_buy_bids.append(pandas.DataFrame(columns=df_buy_bids.columns,data=[[dt_sim_time,ind,float(df_bids['p_buy'].loc[ind]),float(df_bids['q_buy'].loc[ind])]]),ignore_index=True)
        # For V2G
        #if df_bids['q_sell'].loc[ind] > 0.0:
        #    #retail.sell(df_bids['q_sell'].loc[ind],df_bids['p_sell'].loc[ind],gen_name=ind)
        #    df_supply_bids = df_supply_bids.append(pandas.DataFrame(columns=df_supply_bids.columns,data=[[dt_sim_time,ind,float(df_bids['p_sell'].loc[ind]),float(df_bids['q_sell'].loc[ind])]]),ignore_index=True)
    df_bids['active_t-1'] = 0
    return retail,df_supply_bids,df_buy_bids

In [65]:
# Determines if battery should be dispatched and how
def set_EV_by_price(dt_sim_time,df_bids_EV,Pd, df_awarded_bids):
    # Determine activity
    # Make sure that disconneted EVs don't submit bids
    df_bids_EV.at[:,'active_t'] = 0
    df_bids_EV.at[(df_bids_EV['p_buy'] >= Pd) & (df_bids_EV['SOC_t'] < df_bids_EV['SOC_max']),'active_t'] = 1
    #df_bids_EVs.at[(df_bids_EVs['p_sell'] <= Pd) & (df_bids_EVs['SOC_t'] > 0.0),'active_t'] = -1
    # Save to awarded bids
    for EV in df_bids_EV.index:
        EV_number = EV
        SOC = df_bids_EV['SOC_t'].loc[EV] #this is SOC at the beginning of the period t
        active = df_bids_EV['active_t'].loc[EV] #this is activity in t
        if active == 1:
            q_bid = df_bids_EV['q_buy'].loc[EV]
            p_bid = df_bids_EV['p_buy'].loc[EV]
            df_awarded_bids = df_awarded_bids.append(pandas.DataFrame(columns=df_awarded_bids.columns,data=[[dt_sim_time,EV,float(p_bid),float(q_bid),'D']]),ignore_index=True)
        # Relevant for V2G
        #elif active == -1:
        #    q_bid = df_bids_EV['q_sell'].loc[EV]
        #    p_bid = df_bids_EV['p_sell'].loc[EV]
        #    df_awarded_bids = df_awarded_bids.append(pandas.DataFrame(columns=df_awarded_bids.columns,data=[[dt_sim_time,battery,float(p_bid),float(q_bid),'S']]),ignore_index=True)
    return df_bids_EV,df_awarded_bids

## Initialize objects and data

In [66]:
# Initialize market parameters
start = datetime.datetime( 2016, 7, 1,17, 0)
end = datetime.datetime( 2016, 7, 1, 20, 0)
interval = 300 # interval of market operation
market_intervals = pandas.date_range(start,end,freq=str(int(interval/60))+'min')

In [67]:
# Set up EV
cols_EV = ['EV_name','house_name','SOC_max','i_max','v_max','u_max','efficiency','charging_type','k','soc_t','SOC_t','connected','next_event','active_t-1','active_t']
# If EV is currently not connected (connected = 0), next_event is time when the EV gets
#   re-connected/starts charging
next_event = start + pandas.Timedelta(minutes=30)
values = [['EV_1','house_1',40.,100,120,12,1.0,'residential',1.0,0.0,20.,1,next_event,0,0]]
df_EV_state = pandas.DataFrame(columns=cols_EV,data=values)
df_EV_state

,EV_name,house_name,SOC_max,i_max,v_max,u_max,efficiency,charging_type,k,soc_t,SOC_t,connected,next_event,active_t-1,active_t
0,EV_1,house_1,40.0,100,120,12,1.0,residential,1.0,0.0,20.0,1,2016-07-01 17:30:00,0,0


In [68]:
# WS costs
df_WS = pandas.read_csv('glm_generation_Austin/Ercot_LZ_SOUTH.csv',parse_dates=True,index_col=[0]) # For year 2016
df_WS.head()

,DA,RT
2016-01-01 00:00:00,18.22,15.48
2016-01-01 00:05:00,18.22,15.48
2016-01-01 00:10:00,18.22,15.48
2016-01-01 00:15:00,18.22,15.33
2016-01-01 00:20:00,18.22,15.33


## Run market

In [69]:
# Set up df to save results
df_prices = pandas.DataFrame(columns=['clearing_price','clearing_quantity','unresponsive_loads'])
df_buy_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity'])
df_supply_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity'])
df_awarded_bids = pandas.DataFrame(columns=['timestamp','appliance_name','bid_price','bid_quantity','S_D'])

# Run market
for dt_sim_time in market_intervals:
    print(dt_sim_time)
    # Update physical state of charging EVs
    # Simulate arrival and departure
    df_EV_state = update_EV_rnd(dt_sim_time,df_EV_state)
    
    # Setup market
    retail = None # this is a placeholder
    
    # Bid : THIS IS WHERE YOU SHOULD IMPLEMENT AND TEST BIDDING FUNCTIONS
    df_EV_state = bid_simple(dt_sim_time,df_EV_state,df_WS,df_prices)
    #bid_costminizing(dt_sim_time,df_EV_state,df_WS,df_prices)
    #bid_optimal_autarky(dt_sim_time,df_EV_state,df_WS,df_prices)
    retail,df_supply_bids,df_buy_bids = submit_bids_EV(dt_sim_time,retail,df_EV_state,df_supply_bids,df_buy_bids)
    
    # Clear market and save price
    Pd = df_WS['RT'].loc[dt_sim_time:dt_sim_time].iloc[-1]
    Qd = df_EV_state['q_buy'].loc[df_EV_state['p_buy'] >= Pd].sum()
    # For V2G
    #Qd = max(df_EV_state['q_sell'].loc[df_EV_state['p_sell'] <= Pd].sum(),df_EV_state['q_buy'].loc[df_EV_state['p_buy'] >= Pd].sum())
    unresp_load = 0.0
    df_temp = pandas.DataFrame(index=[dt_sim_time],columns=['clearing_price','clearing_quantity','unresponsive_loads'],data=[[Pd,Qd,unresp_load]])
    df_prices = df_prices.append(df_temp)
    
    # Dispatch EV
    df_bids_EV,df_awarded_bids = set_EV_by_price(dt_sim_time,df_EV_state,Pd,df_awarded_bids)

2016-07-01 17:00:00
2016-07-01 17:05:00
2016-07-01 17:10:00
2016-07-01 17:15:00
2016-07-01 17:20:00
2016-07-01 17:25:00
2016-07-01 17:30:00
2016-07-01 17:35:00
2016-07-01 17:40:00
2016-07-01 17:45:00
2016-07-01 17:50:00
2016-07-01 17:55:00
2016-07-01 18:00:00
2016-07-01 18:05:00
2016-07-01 18:10:00
2016-07-01 18:15:00
2016-07-01 18:20:00
2016-07-01 18:25:00
2016-07-01 18:30:00
2016-07-01 18:35:00
2016-07-01 18:40:00
2016-07-01 18:45:00
2016-07-01 18:50:00
2016-07-01 18:55:00
2016-07-01 19:00:00
2016-07-01 19:05:00
2016-07-01 19:10:00
2016-07-01 19:15:00
2016-07-01 19:20:00
2016-07-01 19:25:00
2016-07-01 19:30:00
2016-07-01 19:35:00
2016-07-01 19:40:00
2016-07-01 19:45:00
2016-07-01 19:50:00
2016-07-01 19:55:00
2016-07-01 20:00:00


## Visualize market results

In [70]:
# buy bids
df_buy_bids

,timestamp,appliance_name,bid_price,bid_quantity
0,2016-07-01 17:00:00,0,31.507407,12.0
1,2016-07-01 17:05:00,0,31.136792,12.0
2,2016-07-01 17:10:00,0,30.890755,12.0
3,2016-07-01 17:15:00,0,30.644717,12.0
4,2016-07-01 17:20:00,0,30.398679,12.0
5,2016-07-01 17:25:00,0,30.152642,12.0
6,2016-07-01 17:30:00,0,29.906604,12.0
7,2016-07-01 17:35:00,0,29.660566,12.0
8,2016-07-01 17:40:00,0,29.414528,12.0
9,2016-07-01 17:45:00,0,29.168491,12.0


In [71]:
# supply bids
df_supply_bids

,timestamp,appliance_name,bid_price,bid_quantity


In [72]:
# awarded bids
df_awarded_bids

,timestamp,appliance_name,bid_price,bid_quantity,S_D
0,2016-07-01 17:00:00,0,31.507407,12.0,D
1,2016-07-01 17:15:00,0,30.644717,12.0,D
2,2016-07-01 18:45:00,0,26.904815,12.0,D
3,2016-07-01 18:50:00,0,26.659623,12.0,D
4,2016-07-01 18:55:00,0,26.501887,12.0,D
5,2016-07-01 19:30:00,0,25.116731,12.0,D
6,2016-07-01 19:35:00,0,24.932692,12.0,D
7,2016-07-01 19:40:00,0,24.748654,12.0,D
8,2016-07-01 19:45:00,0,24.564615,12.0,D
9,2016-07-01 19:50:00,0,24.380577,12.0,D


In [73]:
# market result
df_prices

,clearing_price,clearing_quantity,unresponsive_loads
2016-07-01 17:00:00,31.40,12.0,0.0
2016-07-01 17:05:00,31.40,0.0,0.0
2016-07-01 17:10:00,31.40,0.0,0.0
2016-07-01 17:15:00,30.61,12.0,0.0
2016-07-01 17:20:00,30.61,0.0,0.0
2016-07-01 17:25:00,30.61,0.0,0.0
2016-07-01 17:30:00,30.02,0.0,0.0
2016-07-01 17:35:00,30.02,0.0,0.0
2016-07-01 17:40:00,30.02,0.0,0.0
2016-07-01 17:45:00,30.27,0.0,0.0


## Calculate EV charging cost

In [78]:
calculate_EV_cost()

NameError: name 'profit' is not defined

In [79]:
visualize_EV_dispatch()